In [1]:
import pandas as pd
import datetime as dt
import yfinance as yf
import sqlalchemy as sqla
import pandas_market_calendars as mcal

# local modules
import sys
sys.path.append("../")
import lib

In [2]:
# set query parameters
tickers = ["VOO","VIOO","VFVA","VFMO","VFQY","VFMV"]
current_date = str(dt.date.today())
current_time = dt.datetime.now().time()

# set market calendar
nyse_cal  = mcal.get_calendar("NYSE")
valid_mkt_days = [str(t.date()) for t in nyse_cal.valid_days(current_date,current_date)]

In [ ]:
# run only when market has been open
if current_date in valid_mkt_days:

    #  ensure we're past market close (5pm central)
    if current_time >= dt.time(hour=17):

        # download today's data
        ohlc = yf.download(tickers=tickers,period="1d") 

        # stack data and reformat headers
        df = ohlc.stack().reset_index().rename({"level_1":"ticker"},axis=1)
        df.columns= [c.replace(" ","_").lower() for c in df.columns]

        # save data locally
        df.to_csv("./data/prices_{}".format(current_date.replace("-","")))

In [3]:
# download today's data
ohlc = yf.download(tickers=tickers,start="2020-01-01",end="2023-05-18") 

# stack data and reformat headers
df = ohlc.stack().reset_index().rename({"level_1":"ticker"},axis=1)
df.columns= [c.replace(" ","_").lower() for c in df.columns]

df.tail()

[*********************100%***********************]  6 of 6 completed


,date,ticker,adj_close,close,high,low,open,volume
5095,2023-05-17,VFMV,97.360001,97.360001,97.500000,96.400002,96.400002,5900
5096,2023-05-17,VFQY,106.818001,106.818001,106.910004,105.510002,105.510002,3400
5097,2023-05-17,VFVA,94.190002,94.190002,94.269997,92.290001,92.290001,93100
5098,2023-05-17,VIOO,86.150002,86.150002,86.330002,84.339996,84.720001,123600
5099,2023-05-17,VOO,381.450012,381.450012,382.089996,377.279999,378.850006,3222600


In [4]:
# save data to postgres
psql = lib.data.create_psql_eng(lib.creds.psql2)
df.to_sql(name="etfs",schema="prices",if_exists="append",con=psql,index=False)
psql.close()